# Ex-10 Salary Slab Prediction Using Classification Part-1

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv('/content/Salary_data.csv')

In [3]:
census.head()

age          workclass  ...  native_country  income_bracket
0   39          State-gov  ...   United-States           <=50K
1   50   Self-emp-not-inc  ...   United-States           <=50K
2   38            Private  ...   United-States           <=50K
3   53            Private  ...   United-States           <=50K
4   28            Private  ...            Cuba           <=50K

[5 rows x 14 columns]

In [4]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
def label_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

In [6]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [7]:
census.head()

age          workclass  ...  native_country  income_bracket
0   39          State-gov  ...   United-States               0
1   50   Self-emp-not-inc  ...   United-States               0
2   38            Private  ...   United-States               0
3   53            Private  ...   United-States               0
4   28            Private  ...            Cuba               0

[5 rows x 14 columns]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
x_data = census.drop('income_bracket',axis=1)

In [10]:
y_labels = census['income_bracket']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_labels, test_size=0.3, random_state=101)

In [12]:
X_train.head()

age   workclass  ... hours_per_week  native_country
20895   22     Private  ...             28   United-States
3384    47     Private  ...             40   United-States
1832    46   Local-gov  ...             24   United-States
18919   46   State-gov  ...             48   United-States
31685   60     Private  ...             40   United-States

[5 rows x 13 columns]

In [13]:
y_train.head()

20895    0
3384     1
1832     0
18919    0
31685    0
Name: income_bracket, dtype: int64

In [14]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [21]:
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

In [22]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender",["Female","Male"])

In [23]:
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [24]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [25]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,age,
            education_num,capital_gain,capital_loss,hours_per_week]

In [26]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [27]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpdc_53byi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [28]:
model.train(input_fn=input_func, steps=10000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpdc_53byi/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving c

In [31]:
pred_fun = tf.estimator.inputs.pandas_input_fn(x= X_test, batch_size=len(X_test), shuffle=False)

In [30]:
prediction = list(model.predict(input_fn=pred_fun))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpdc_53byi/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [32]:
prediction[0]

{'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'logistic': array([0.3016407], dtype=float32),
 'logits': array([-0.83949727], dtype=float32),
 'probabilities': array([0.6983593, 0.3016407], dtype=float32)}

In [33]:
final_preds = []
for pred in prediction:
    final_preds.append(pred['class_ids'][0])

In [ ]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.88      0.92      0.90      7436
          1       0.72      0.62      0.66      2333

avg / total       0.85      0.85      0.85      9769

